In [4]:
!pip install wikipedia
!pip install youtube_search


In [1]:
from langchain.llms.openai import OpenAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import YouTubeSearchTool
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

import os

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="{YOUR_LANGCHAIN_APIKEY}"
os.environ["LANGCHAIN_PROJECT"]="{YOUR_LANGSMITH_PROJECT}"
os.environ["OPENAI_API_KEY"] = "{YOUR_OPENAI_KEY}"
os.environ["SERPER_API_KEY"] = "YOU_SERPER_APIKEY"

model = OpenAI()

search = GoogleSerperAPIWrapper()
youtube = YouTubeSearchTool()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [
    Tool(
        name="Google Search",
        func=search.run,
        description="useful for when you need to ask with search",
        verbose=True
    ),
    Tool(
        name="Youtube Search",
        func=youtube.run,
        description="useful for when the user explicitly asks you to look on Youtube",
        verbose=True
    ),
    Tool(
       name="Wikipedia Search",
       func=wikipedia.run,
       description="Useful when users request historical moments.",
   )
    
]

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

agent = create_react_agent(model,tools,prompt) 
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)
response = agent_executor.invoke({"input": "Tell me about Korean war with details ?"})

/Users/terrycho/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(




> Entering new AgentExecutor chain...
 This is a specific question about a historical event, so using Wikipedia Search may be the best option.
Action: Wikipedia Search
Action Input: "Korean War"Page: Korean War
Summary: The Korean War was fought between North Korea and South Korea from 1950 to 1953. The war began on 25 June 1950 when North Korea invaded South Korea. The war ceased with an armistice on 27 July 1953. North Korea was supported by China and the Soviet Union while South Korea was supported by the United States and the United Nations (UN).
In 1910, the Empire of Japan annexed Korea as a colony for 35 years until its surrender at the end of World War II on 15 August 1945. The United States and the Soviet Union liberated and then divided Korea along the 38th parallel into two zones of occupation. The Soviets administered the northern zone, and the Americans administered the southern zone. In 1948, as a result of Cold War tensions, the occupation zones became two sovereign st

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: ` This provides a thorough overview of the events during the Korean War. The final answer is that the Korean War was fought between North Korea and South Korea from 1950 to 1953, resulting in an armistice on 27 July 1953 and the division of Korea into two sovereign states. It was a major conflict of the Cold War and caused significant destruction and loss of life.`

In [9]:
agent_executor.invoke({"input": "Can you recommend videos about Korean war ?"})



> Entering new AgentExecutor chain...
 I should use Youtube Search since the user is explicitly asking for videos.
Action: Youtube Search
Action Input: Korean War['https://www.youtube.com/watch?v=yxaegqvl4aE&pp=ygUKS29yZWFuIFdhcg%3D%3D', 'https://www.youtube.com/watch?v=ViVGj58kt34&pp=ygUKS29yZWFuIFdhcg%3D%3D']I now know the final answer
Final Answer: ['https://www.youtube.com/watch?v=yxaegqvl4aE&pp=ygUKS29yZWFuIFdhcg%3D%3D', 'https://www.youtube.com/watch?v=ViVGj58kt34&pp=ygUKS29yZWFuIFdhcg%3D%3D']

> Finished chain.


{'input': 'Can you recommend videos about Korean war ?',
 'output': "['https://www.youtube.com/watch?v=yxaegqvl4aE&pp=ygUKS29yZWFuIFdhcg%3D%3D', 'https://www.youtube.com/watch?v=ViVGj58kt34&pp=ygUKS29yZWFuIFdhcg%3D%3D']",
 'intermediate_steps': [(AgentAction(tool='Youtube Search', tool_input='Korean War', log=' I should use Youtube Search since the user is explicitly asking for videos.\nAction: Youtube Search\nAction Input: Korean War'),
   "['https://www.youtube.com/watch?v=yxaegqvl4aE&pp=ygUKS29yZWFuIFdhcg%3D%3D', 'https://www.youtube.com/watch?v=ViVGj58kt34&pp=ygUKS29yZWFuIFdhcg%3D%3D']")]}